# Final Project

## Predict whether a mammogram mass is benign or malignant

We'll be using the "mammographic masses" public dataset from the UCI repository (source: https://archive.ics.uci.edu/ml/datasets/Mammographic+Mass)

This data contains 961 instances of masses detected in mammograms, and contains the following attributes:


   1. BI-RADS assessment: 1 to 5 (ordinal)  
   2. Age: patient's age in years (integer)
   3. Shape: mass shape: round=1 oval=2 lobular=3 irregular=4 (nominal)
   4. Margin: mass margin: circumscribed=1 microlobulated=2 obscured=3 ill-defined=4 spiculated=5 (nominal)
   5. Density: mass density high=1 iso=2 low=3 fat-containing=4 (ordinal)
   6. Severity: benign=0 or malignant=1 (binominal)
   
BI-RADS is an assesment of how confident the severity classification is; it is not a "predictive" attribute and so we will discard it. The age, shape, margin, and density attributes are the features that we will build our model with, and "severity" is the classification we will attempt to predict based on those attributes.

Although "shape" and "margin" are nominal data types, which sklearn typically doesn't deal with well, they are close enough to ordinal that we shouldn't just discard them. The "shape" for example is ordered increasingly from round to irregular.

A lot of unnecessary anguish and surgery arises from false positives arising from mammogram results. If we can build a better way to interpret them through supervised machine learning, it could improve a lot of lives.

## Your assignment

Apply several different supervised machine learning techniques to this data set, and see which one yields the highest accuracy as measured with K-Fold cross validation (K=10). Apply:

* Decision tree
* Random forest
* KNN
* Naive Bayes
* SVM
* Logistic Regression
* And, as a bonus challenge, a neural network using Keras.

The data needs to be cleaned; many rows contain missing data, and there may be erroneous data identifiable as outliers as well.

Remember some techniques such as SVM also require the input data to be normalized first.

Many techniques also have "hyperparameters" that need to be tuned. Once you identify a promising approach, see if you can make it even better by tuning its hyperparameters.

I was able to achieve over 80% accuracy - can you beat that?

Below I've set up an outline of a notebook for this project, with some guidance and hints. If you're up for a real challenge, try doing this project from scratch in a new, clean notebook!


## Let's begin: prepare your data

Start by importing the mammographic_masses.data.txt file into a Pandas dataframe (hint: use read_csv) and take a look at it.

In [2]:
#This is my work for the final project in skillshare with Frank Kane. 
#The comments are my thoughts and mistakes were left so that I could go back and work on them later. 
import pandas as pd

mammodata = pd.read_csv('mammographic_masses.data.txt')
mammodata.head()

,5,67,3,5.1,3.1,1
0,4,43,1,1,?,1
1,5,58,4,5,3,1
2,4,28,1,1,3,0
3,5,74,1,5,?,1
4,4,65,1,?,3,0


Make sure you use the optional parmaters in read_csv to convert missing data (indicated by a ?) into NaN, and to add the appropriate column names (BI_RADS, age, shape, margin, density, and severity):

In [3]:
mammonames = ['BI_RADS', 'age', 'shape', 'margin', 'density', 'severity']
mammodata = pd.read_csv('mammographic_masses.data.txt', na_values='?', names=mammonames)
mammodata.head()

,BI_RADS,age,shape,margin,density,severity
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


Evaluate whether the data needs cleaning; your model is only as good as the data it's given. Hint: use describe() on the dataframe.

In [4]:
mammodata.describe()

,BI_RADS,age,shape,margin,density,severity
count,959.000000,956.000000,930.000000,913.000000,885.000000,961.000000
mean,4.348279,55.487448,2.721505,2.796276,2.910734,0.463059
std,1.783031,14.480131,1.242792,1.566546,0.380444,0.498893
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,45.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


There are quite a few missing values in the data set. Before we just drop every row that's missing data, let's make sure we don't bias our data in doing so. Does there appear to be any sort of correlation to what sort of data has missing fields? If there were, we'd have to try and go back and fill that data in.

If the missing data seems randomly distributed, go ahead and drop rows with missing data. Hint: use dropna().

In [5]:
mdata = mammodata.dropna()
mdata.describe()

,BI_RADS,age,shape,margin,density,severity
count,830.000000,830.000000,830.000000,830.000000,830.000000,830.000000
mean,4.393976,55.781928,2.781928,2.813253,2.915663,0.485542
std,1.888371,14.671782,1.242361,1.567175,0.350936,0.500092
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,46.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


Next you'll need to convert the Pandas dataframes into numpy arrays that can be used by scikit_learn. Create an array that extracts only the feature data we want to work with (age, shape, margin, and density) and another array that contains the classes (severity). You'll also need an array of the feature name labels.

In [6]:
import numpy as np
features = np.array(mdata[['age', 'shape', 'margin', 'density']])
featurenames = np.array(['age', 'shape', 'margin', 'density'])
label = np.array(mdata['severity'])
features[:5]


array([[67.,  3.,  5.,  3.],
       [58.,  4.,  5.,  3.],
       [28.,  1.,  1.,  3.],
       [57.,  1.,  5.,  3.],
       [76.,  1.,  4.,  3.]])

Some of our models require the input data to be normalized, so go ahead and normalize the attribute data. Hint: use preprocessing.StandardScaler().

In [7]:
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

scale = StandardScaler()
#m = featurenames
Xscaled = scale.fit_transform(features) #was once X then for SVM turned this into Xscaled, idk what happened in between
y = label
print(Xscaled)
#print(y)

[[ 0.7650629   0.17563638  1.39618483  0.24046607]
 [ 0.15127063  0.98104077  1.39618483  0.24046607]
 [-1.89470363 -1.43517241 -1.157718    0.24046607]
 ...
 [ 0.56046548  0.98104077  1.39618483  0.24046607]
 [ 0.69686376  0.98104077  1.39618483  0.24046607]
 [ 0.42406719  0.17563638  0.11923341  0.24046607]]


## Decision Trees

Before moving to K-Fold cross validation and random forests, start by creating a single train/test split of our data. Set aside 75% for training, and 25% for testing.

In [8]:
from sklearn.model_selection import train_test_split
np.random.seed(1)
x_train, x_test, y_train, y_test = train_test_split(Xscaled, y, test_size=0.25, random_state=1)
#y_train

Now create a DecisionTreeClassifier and fit it to your training data.

In [9]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(random_state=1)
clf = clf.fit(x_train, y_train)

Display the resulting decision tree.

In [12]:
from IPython.display import Image  
from sklearn.externals.six import StringIO  
import pydotplus
import graphviz

dot_data = StringIO()  
tree.export_graphviz(clf, out_file=dot_data,  
                         feature_names=featurenames)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())  
#graphviz does not work on my personal computer for some reason

InvocationException: GraphViz's executables not found

Measure the accuracy of the resulting decision tree model using your test data.

In [10]:
#predictions = clf.predict(x_test)
clf.score(x_test, y_test)

0.7355769230769231

Now instead of a single train/test split, use K-Fold cross validation to get a better measure of your model's accuracy (K=10). Hint: use model_selection.cross_val_score

In [11]:
from sklearn.model_selection import cross_val_score
clf = tree.DecisionTreeClassifier(random_state=1)
scores = cross_val_score(clf, Xscaled, y, cv=10)
print(scores.mean())

0.7373123154639465


Now try a RandomForestClassifier instead. Does it perform better?

In [13]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=20, random_state=1)
clf = clf.fit(Xscaled, y)
score = cross_val_score(clf, Xscaled, y, cv=10)
print(score.mean())

#predictions on a known malignant and known begnin test. begnin varies between 1 and 0. Performs about the same
print(clf.predict([[50,4,5,3]]))
print(clf.predict([[25,1,1,3]]))

0.7649087639757636
[1]
[0]


## SVM

Next try using svm.SVC with a linear kernel. How does it compare to the decision tree?

In [15]:
from sklearn import svm, datasets

C = 1.0 #hyperparameter
svc = svm.SVC(kernel='linear', C=C, gamma='scale').fit(Xscaled, y)
print(svc.score(x_test, y_test)) #different from cross value score
sco = cross_val_score(svc, Xscaled, y, cv=10)
print(sco.mean())

0.7740384615384616
0.7964988875362076


In [21]:
%matplotlib inline
from pylab import *
from sklearn.preprocessing import MinMaxScaler

#(X, y) = createClusteredData(100, 5)

plt.figure(figsize=(8, 6))
plt.scatter(X[:,0], X[:,1], c=y.astype(np.float))
plt.show()

scaling = MinMaxScaler(feature_range=(-1,1)).fit(X)
X = scaling.transform(X)

plt.figure(figsize=(8, 6))
plt.scatter(X[:,0], X[:,1], c=y.astype(np.float))
plt.show()
#already scaled down so the graphs are the same

NameError: name 'X' is not defined

<Figure size 576x432 with 0 Axes>

## KNN
How about K-Nearest-Neighbors? Hint: use neighbors.KNeighborsClassifier - it's a lot easier than implementing KNN from scratch like we did earlier in the course. Start with a K of 10. K is an example of a hyperparameter - a parameter on the model itself which may need to be tuned for best results on your particular data set.

In [18]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(Xscaled,y)
print(knn.score(x_test, y_test))
knnscore = cross_val_score(knn, Xscaled, y, cv=10)
print(knnscore.mean())

0.7788461538461539
0.7854795488574507


Choosing K is tricky, so we can't discard KNN until we've tried different values of K. Write a for loop to run KNN with K values ranging from 1 to 50 and see if K makes a substantial difference. Make a note of the best performance you could get out of KNN.

In [19]:
KNN = {}
for n in range(1,51):
    kn = KNeighborsClassifier(n_neighbors=n)
    kn.fit(Xscaled,y)
    score = cross_val_score(kn, Xscaled, y, cv=10)
    score = score.mean()
    KNN[n] = score
    
import operator
#print(KNN)
maximum = max(KNN.items(), key=operator.itemgetter(1))[0]
print('Best Performance:', maximum, ':', KNN[maximum]) #n=7 is the best K value, but apparently changes if something else alters

Best Performance: 7 : 0.7940595133145824


## Naive Bayes

Now try naive_bayes.MultinomialNB. How does its accuracy stack up? Hint: you'll need to use MinMaxScaler to get the features in the range MultinomialNB requires.

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
scaling = MinMaxScaler().fit(Xscaled) #use original unscaled X for NB classifier and scale with minmax scaler
X = scaling.transform(Xscaled)
NBscores = cross_val_score(classifier, X, y, cv=10)
print(NBscores.mean())
#not too bad for the naivety of the technique

0.7844055665169388


## Revisiting SVM

svm.SVC may perform differently with different kernels. The choice of kernel is an example of a "hyperparamter." Try the rbf, sigmoid, and poly kernels and see what the best-performing kernel is. Do we have a new winner?

In [23]:
C = 1.0 #hyperparameter
svc = svm.SVC(kernel='poly', degree=5, C=C, gamma='scale').fit(X,y) #changed to best degree
#svc.score(x_test, y_test)
svcscore = cross_val_score(svc, Xscaled, y, cv=10)
print(svcscore.mean())

0.7615416369309992


In [24]:
C = 1.0 #hyperparameter
svc = svm.SVC(kernel='rbf', C=C, gamma='scale').fit(X,y)
svc.score(x_test, y_test) #best out of the three
svcscore = cross_val_score(svc, Xscaled, y, cv=10)
print(svcscore.mean())

0.8023928466479158


In [25]:
C = 1.0 #hyperparameter
svc = svm.SVC(kernel='sigmoid', C=C, gamma='scale').fit(X,y)
svc.score(x_test, y_test)
svcscore = cross_val_score(svc, Xscaled, y, cv=10)
print(svcscore.mean())

0.7374711389110449


## Logistic Regression

We've tried all these fancy techniques, but fundamentally this is just a binary classification problem. Try Logisitic Regression, which is a simple way to tackling this sort of thing.

In [74]:
est = sm.OLS(y, X).fit()

est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.216
Model:                            OLS   Adj. R-squared (uncentered):              0.212
Method:                 Least Squares   F-statistic:                              56.73
Date:                Wed, 06 Nov 2019   Prob (F-statistic):                    2.61e-42
Time:                        12:37:36   Log-Likelihood:                         -777.15
No. Observations:                 830   AIC:                                      1562.
Df Residuals:                     826   BIC:                                      1581.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1181      0.024      4.956      0.000       0.071       0.165
x2             0.1365      0.032      4.261      0.000       0.074       0.199
x3             0.1365      0.033      4.159      0.000       0.072       0.201
x4             0.0010      0.022      0.045      0.964      -0.042       0.043
==============================================================================
Omnibus:                        8.074   Durbin-Watson:                   0.798
Prob(Omnibus):                  0.018   Jarque-Bera (JB):                8.201
Skew:                          -0.243   Prob(JB):                       0.0166
Kurtosis:                       2.958   Cond. No.                         2.83
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
%matplotlib inline
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns


slope, intercept, r_value, p_value, std_err = stats.linregress(x_test, y_test)
r_value ** 2 #clearly not working for me because X is in 4D and y is in 1D, but the arrays themselves are indeed the same size
#not sure if I even learned this yet...?

ValueError: all the input array dimensions except for the concatenation axis must match exactly

## Neural Networks

As a bonus challenge, let's see if an artificial neural network can do even better. You can use Keras to set up a neural network with 1 binary output neuron and see how it performs. Don't be afraid to run a large number of epochs to train the model if necessary.

## Do we have a winner?

Which model, and which choice of hyperparameters, performed the best? Feel free to share your results!

# My Answer
The best I could get is 81%. The best ones that I could get to work were the KNN which goes between 81-86%, and the SVM with C=4 (because 4 is the highest dimension and not sure if I can go over that or not) and kernel=rbf. Decision trees is the lowest with 73%.